# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [1]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
  
# metadata 
print(adult.metadata) 
  
# variable information 
print(adult.variables) 

{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Mon Aug 07 2023', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': 'Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the following conditions: ((AAG

In [4]:
import pandas as pd
# Define your column names if necessary
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]

# Load data from a URL
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
adult_dt = pd.read_csv(data_url, header=None, names=columns)

# Example usage, assuming 'income' is a string '>50K' or '<=50K'
adult_dt['income'] = (adult_dt['income'].str.strip() == '>50K').astype(int)

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [5]:
# Remove Holand-Netherlands that is causing issues when running cross validation
adult_dt = adult_dt[adult_dt['native-country']!=' Holand-Netherlands']

X = adult_dt.drop('income',axis=1)
Y = adult_dt['income']

from sklearn.model_selection import train_test_split 
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler

# Identify numerical and categorical columns
num_cols = ['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']

cat_cols = ['workclass','education','marital-status','occupation','relationship','race','sex','native-country']


# Creating a pipeline of tasks

    # num_transforms functions
num_transforms = Pipeline(steps=[
    ('KNNimputer', KNNImputer(n_neighbors = 7,weights='distance')),
    ('scaler', RobustScaler())
])
    # cat_transforms functions
cat_transforms= Pipeline(steps=[
    ('simpleimputer',SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist',drop='if_binary'))
])

    # column transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', num_transforms, num_cols), 
        ('cat_transforms', cat_transforms, cat_cols)   
    ]
)

preprocessor

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('KNNimputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='infrequent_if_exist'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.ensemble import RandomForestClassifier

# Identify numerical and categorical columns
num_cols = ['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']

cat_cols = ['workclass','education','marital-status','occupation','relationship','race','sex','native-country']


# Creating a pipeline of tasks

    # num_transforms functions
num_transforms = Pipeline(steps=[
    ('KNNimputer', KNNImputer(n_neighbors = 7,weights='distance')),
    ('scaler', RobustScaler())
])
    # cat_transforms functions
cat_transforms= Pipeline(steps=[
    ('simpleimputer',SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist',drop='if_binary'))
])

    # column transformers # should be name of columns as a list
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', num_transforms, num_cols), 
        ('cat_transforms', cat_transforms, cat_cols)   
    ]
)


# Create  pipeline
pipeline = Pipeline(steps=[
    ('Preprocessing', preprocessor)
    ,('Forest',RandomForestClassifier())

])

pipeline

Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('KNNimputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='infrequent_if_exist'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('Forest', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [8]:
pipeline.fit(X_train,Y_train)
from sklearn.model_selection import cross_validate
scoring = ['neg_log_loss', 'roc_auc', 'accuracy','balanced_accuracy']


res_simple_dict = cross_validate(pipeline, X_train, Y_train, cv = 5, scoring = scoring)

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [9]:
res_simple = pd.DataFrame(res_simple_dict).assign(experiment = 1)
res_simple.sort_values(by='test_neg_log_loss')

,fit_time,score_time,test_neg_log_loss,test_roc_auc,test_accuracy,test_balanced_accuracy,experiment
4,9.862329,0.217501,-0.367434,0.902187,0.855419,0.769994,1
2,9.777431,0.214722,-0.366102,0.901999,0.855419,0.776059,1
0,9.882269,0.229217,-0.361556,0.905298,0.854135,0.780657,1
3,10.129063,0.273897,-0.358864,0.909371,0.859368,0.780218,1
1,9.750852,0.226952,-0.326531,0.910365,0.850406,0.772286,1


Calculate the mean of each metric. 

In [10]:
res_simple.mean()

fit_time                  9.880389
score_time                0.232458
test_neg_log_loss        -0.356097
test_roc_auc              0.905844
test_accuracy             0.854949
test_balanced_accuracy    0.775843
experiment                1.000000
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [12]:
import sklearn.metrics
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Perform pipeline functions on the X data sets
# Y_pred_train = pipeline.predict(X_train)
Y_pred_test = pipeline.predict(X_test)
# Y_proba_train = pipeline.predict_proba(X_train)
Y_proba_test = pipeline.predict_proba(X_test)

# create a dictionary to store model results/performance metrics
res = {
    'neg_log_loss': -log_loss(Y_test, Y_proba_test),
    'roc_auc': roc_auc_score(Y_test, Y_proba_test[:, 1]),
    'accuracy': accuracy_score(Y_test, Y_pred_test),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred_test)
}

# Display the results
print(res)


{'neg_log_loss': -0.40568178228620816, 'roc_auc': 0.9000349114451164, 'accuracy': 0.8551392301392301, 'balanced_accuracy': 0.7765784837902154}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

This code divides data into two categories for binary classification. It ensures analysis simplicity, accuracy, and ROC applicability for binary targets, estimating income probabilities over $50,000 and ensuring target variable suitability for scikit-learn tasks.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/scaling_to_production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.